In [ ]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [9]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

In [10]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [11]:
import requests
from bs4 import BeautifulSoup
url = 'https://www.google.co.kr'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

In [12]:
driver.get(url)
time.sleep(1)

In [13]:
search_box = driver.find_element(by=By.NAME, value='q')
search_box.send_keys('ChromDriver')
search_box.send_keys(Keys.ENTER)
time.sleep(2)

In [14]:
divs = driver.find_elements(by=By.CSS_SELECTOR, value='#search .g')

In [15]:
lines = []
for div in divs:
    try:
        title = div.find_element(by=By.CSS_SELECTOR, value='.LC20lb.MBeuO.DKV0Md').text
        content = div.find_element(By.CSS_SELECTOR, ".VwiC3b.yXK7lf.lVm3ye").text.strip()
    except:
        continue
    lines.append([title, content])

In [17]:
import pandas as pd
df = pd.DataFrame(lines, columns=['제목','내용'])
df

,제목,내용
0,ChromeDriver - WebDriver for Chrome,ChromeDriver. WebDriver is an open source tool...
1,ChromeDriver - WebDriver for Chrome,ChromeDriver. WebDriver is an open source tool...
2,chromedriver,An NPM wrapper for Selenium ChromeDriver. Buil...
3,Chrome for Testing availability,Chrome for Testing availability. This page lis...
4,Selenium.WebDriver.ChromeDriver 120.0.6099.7100,This NuGet package installs Chrome Driver (Win...
5,chromedriver-py,downloads and installs the latest chromedriver...
6,Seeking Advice for Chrome Browser 119.0.6045.1...,"Nov 16, 2023 — I'm facing an issue with Chrome..."
7,ChromeDriver · SeleniumHQ/selenium Wiki,A browser automation framework and ecosystem. ...
8,Chrome drivers,"Index of / ; [DIR], 2.14, - ; [DIR], 2.15, - ;..."
